In [4]:
pip install tenacity


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [157]:
import requests
from bs4 import BeautifulSoup
from tenacity import retry, stop_after_attempt,wait_exponential
import urllib
import time
import numpy as np
import pandas as pd

#tenacityのライブラリに関する記事は下記
#https://zenn.dev/taroman_zenn/articles/dd0b33a3a37d1e

In [188]:
#Step1必要な情報を取得する

#airdoor上で必要な情報を検索する
#今回はペルソナに合致した下記の条件を設定｜ペルソナ：https://www.dropbox.com/scl/fi/pdo0x2sm6es6n76sek2i7/Tech0_STEP3-1_-_v1.0.pdf?rlkey=2attz3z14yyoanwm53spxk6t1&dl=0
#検索条件：東京都内、20万円以下（管理費・共益費込み）、2K以上の間取り、駅徒歩10分以内、バストイレ別、室内洗濯機置場
#取得したURL：https://airdoor.jp/list?si=d-131016-131024-131032-131041-131059-131067-131075-131083-131091-131105-131113-131121-131130-131148-131156-131164-131172-131181-131199-131202-131211-131229-131237&ur=200000&iaf=1&uf=10&fp=d-2_k-2_dk-2_ldk-3_k-3_dk-3_ldk-4_more&ca=d-10-43

#複数ページの情報をまとめて取得するリストを作成
data_samples = []

#スクレイピングするページ数を指定
#出力されたページ数16を上回るように設定する
max_page = 20

#airdoorで自分が調べたい地域を指定して検索し、出力したページのURL
URL = 'https://airdoor.jp/list?si=d-131016-131024-131032-131041-131059-131067-131075-131083-131091-131105-131113-131121-131130-131148-131156-131164-131172-131181-131199-131202-131211-131229-131237&ur=200000&iaf=1&uf=10&fp=d-2_k-2_dk-2_ldk-3_k-3_dk-3_ldk-4_more&ca=d-10-43&p={:d}'
airdoor_URL ='https://airdoor.jp'

#試行回数3回、リトライ間の初期待機時間10秒、連続するリトライの時間を100秒にする（指数関数的に増やすが、100秒までで制限する）
#Pythonでは、デコレータの引数を|で結合することはできないため、,で指定すること
@retry(stop=stop_after_attempt(3),wait=wait_exponential(multiplier=10, min=10, max=100))
def load_page(URL):
    html = requests.get(URL)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup


In [189]:
# 処理時間を測りたい
start = time.time()
times = []

#関数については下記を参考にすること
#enumerate()関数 https://note.nkmk.me/python-enumerate-start/
#strip()関数 https://note.nkmk.me/python-str-remove-strip/#strip


for page in range(1,max_page+1):
    before = time.time()
    #ページ情報
    soup = load_page(URL.format(page))
    #物件リストを指定
    mother = soup.find_all(class_='PropertyPanel_propertyPanel__MqCpF')
    
    #物件ごとの処理
    for child in mother:

        #建物情報
        data_home = []
        #建物名
        data_home.append(child.find(class_='PropertyPanelBuilding_buildingTitle__NbWmb').text)
        #アクセスと住所、建物情報のための前処理
        building_info = child.find_all(class_='PropertyPanelBuilding_buildingInformationSection__AMRsh')
        accessANDaddress_info = building_info[0] #アクセス情報のみに切り分ける
        arch_info = building_info[1] #建物情報のみに切り分ける
        #アクセス
        access_info_texts = [p.get_text() for p in accessANDaddress_info.find_all('p')[:-1]] # 各要素内のすべての<p>タグを探し、最後のもの（住所情報）を除外する
        if len(access_info_texts) == 2:
                for id, access_info in enumerate(access_info_texts):
                        data_home.append(access_info)
        else:
                data_home.append(access_info)
                data_home.append(None) #カラムを合わせるためにNaNを追加する
        #住所
        address_info_text = accessANDaddress_info.find_all('p')[-1].get_text()
        data_home.append(address_info_text)
        #築年数
        arch_year = arch_info.find_all('p')[0].get_text()
        data_home.append(arch_year)
        #階数
        arch_floor = arch_info.find_all('p')[1].get_text()
        data_home.append(arch_floor)

        #部屋情報
        for room in child.find_all(class_='PropertyPanelRoom_roomItem__3bVhC'):
            data_room = []

            #家賃と管理費
            price_div = room.find(class_='PropertyPanelRoom_rentPrice__HO4Jp')
            price = price_div.contents[0].strip()
            management_cost = price_div.find('span').text.strip()
            remove_chars = str.maketrans('', '', '()')
            management_cost = management_cost.translate(remove_chars)
            data_room.append(price)
            data_room.append(management_cost)
            #敷金と礼金
            costs = room.find_all(class_='PropertyPanelRoom_initialPrices__naYEA')
            for cost_div in costs: #costs が ResultSet オブジェクトであり、find_all('li') メソッドを直接適用できないためこの処理を入れる
                    li_tags = cost_div.find_all('li')
                    #敷金
                    if len(li_tags) > 0:
                            data_room.append(li_tags[0].get_text()) 
                    #礼金
                    if len(li_tags) > 1:
                            data_room.append(li_tags[1].get_text()) 
            #仲介手数料
            div_fee_amount = room.find(class_='PropertyPanelRoom_roomItemRightContent__UzfRX')
            if "PropertyPanelRoom_isRed__HXtHO" in str(div_fee_amount):
                    fee_amount = div_fee_amount.select("span.PropertyPanelRoom_isRed__HXtHO")[0].text
                    if "無料" in fee_amount:
                            fee_amount = "無料"
                    else:
                            pass
            else:
                    fee_amount = div_fee_amount.select("span")[1].text
            data_room.append(fee_amount)
            
            #階と間取りと面積
            spec_div = room.find_all('p')[0]
            specs = spec_div.text.split('/') # divタグのテキストを取得し、'/'で分割します
            for spec in specs:
                    data_room.append(spec.strip())

            #url：roomからURLが抜き出せないので、一旦無視する
        #     get_url = room.find(class_='PropertyPanelRoom_roomItem__3bVhC').get('href')
        #     abs_url = urllib.parse.urljoin(airdoor_URL, get_url)
        #     data_home.append(abs_url)


            #物件情報と部屋情報を結合する
            data_sample = data_home + data_room
            data_samples.append(data_sample)


    #1アクセスごとに1秒休む
    time.sleep(1)

    #進捗確認
    #このページの作業時間を表示
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'総取得件数:{len(data_home)}')
    #作業進捗
    complete_ratio = round(page/max_page*100,3)
    print(f'完了:{complete_ratio}%')
    #作業の残り時間目安を表示
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間:{hour}時間{minute}分{second}秒\n')


# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：',running_all)

総取得件数:6
完了:5.0%
残り時間:0時間1分9秒

総取得件数:6
完了:10.0%
残り時間:0時間0分57秒

総取得件数:6
完了:15.0%
残り時間:0時間0分54秒

総取得件数:6
完了:20.0%
残り時間:0時間0分48秒

総取得件数:6
完了:25.0%
残り時間:0時間0分43秒

総取得件数:6
完了:30.0%
残り時間:0時間0分39秒

総取得件数:6
完了:35.0%
残り時間:0時間0分36秒

総取得件数:6
完了:40.0%
残り時間:0時間0分33秒

総取得件数:6
完了:45.0%
残り時間:0時間0分30秒

総取得件数:6
完了:50.0%
残り時間:0時間0分27秒

総取得件数:6
完了:55.0%
残り時間:0時間0分24秒

総取得件数:6
完了:60.0%
残り時間:0時間0分21秒

総取得件数:6
完了:65.0%
残り時間:0時間0分19秒

総取得件数:6
完了:70.0%
残り時間:0時間0分16秒

総取得件数:6
完了:75.0%
残り時間:0時間0分13秒

総取得件数:6
完了:80.0%
残り時間:0時間0分10秒

総取得件数:6
完了:85.0%
残り時間:0時間0分8秒

総取得件数:6
完了:90.0%
残り時間:0時間0分5秒

総取得件数:6
完了:95.0%
残り時間:0時間0分2秒

総取得件数:6
完了:100.0%
残り時間:0時間0分0秒

総経過時間： 52.387507915496826


In [190]:
df = pd.DataFrame(data_samples)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,【空室14件】 ジェノヴィア池袋本町,東武東上線 下板橋駅 徒歩8分,東武東上線 北池袋駅 徒歩9分,東京都豊島区池袋本町２丁目,2023年8月(新築),7階建,"119,000円","管理費10,000円",無料,無料,無料,603号室,2K,25.95㎡,西
1,【空室14件】 ジェノヴィア池袋本町,東武東上線 下板橋駅 徒歩8分,東武東上線 北池袋駅 徒歩9分,東京都豊島区池袋本町２丁目,2023年8月(新築),7階建,"115,500円","管理費10,000円",無料,無料,無料,103号室,2K,25.95㎡,西
2,【空室14件】 ジェノヴィア池袋本町,東武東上線 下板橋駅 徒歩8分,東武東上線 北池袋駅 徒歩9分,東京都豊島区池袋本町２丁目,2023年8月(新築),7階建,"115,500円","管理費10,000円",無料,無料,無料,105号室,2K,25.95㎡,西
3,【空室1件】 パークアクシス池上,東急池上線 池上駅 徒歩7分,東急多摩川線 武蔵新田駅 徒歩11分,東京都大田区池上７丁目,2015年3月(築8年),5階建,"157,000円","管理費15,000円",15.7万円,無料,無料,306号室,2DK,41.12㎡,南
4,【空室1件】 オープンレジデンシア上野入谷,東京メトロ日比谷線 入谷駅 徒歩6分,つくばエクスプレス 浅草駅 徒歩9分,東京都台東区入谷２丁目,2023年8月(新築),15階建,"189,000円","管理費10,000円",18.9万円,18.9万円,"88,000円",301号室,2LDK,48.1㎡,西
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,【空室1件】 第一金子ビル,東京メトロ千代田線 綾瀬駅 徒歩5分,つくばエクスプレス 青井駅 徒歩12分,東京都足立区綾瀬４丁目,1985年1月(築38年),5階建,"90,000円","管理費5,000円",9.0万円,9.0万円,"64,000円",202号室,2DK,48.6㎡,西
362,【空室1件】 Ｂｅａｎ’ｓ新中野,東京メトロ丸ノ内線 新中野駅 徒歩2分,ＪＲ総武線 中野駅 徒歩14分,東京都中野区中央３丁目,2002年1月(築21年),3階建,"138,000円","管理費6,000円",13.8万円,13.8万円,"88,000円",207号室,2DK,38㎡,西
363,【空室1件】 ガーデンハイツたまがわ Ａ棟,東急大井町線 尾山台駅 徒歩20分,東急東横線 田園調布駅 徒歩18分,東京都世田谷区玉堤１丁目,1988年9月(築35年),2階建,"135,000円",管理費0円,13.5万円,13.5万円,"88,000円",A3号室,2LDK,57.96㎡,南西
364,【空室1件】 水元ハウス,ＪＲ常磐線 金町駅 バス8分 水元総合スポーツセンター入口下車 徒歩2分,京成金町線 京成金町駅 バス8分 水元総合スポーツセンター入口下車 徒歩2分,東京都葛飾区水元３丁目,1971年1月(築52年),2階建,"133,000円","管理費5,000円",26.6万円,13.3万円,"88,000円",1号室,2LDK,53㎡,北


In [191]:
#Step2データを整形する

#リスト内包表記を使用し、data_samplesリストの各要素からアイテムを取得し、新しいリスト split_data_samplesへ格納する
#item[0]となっているのは、タプル内の要素が1つのリストのみであるため＝([],)となっている

# DataFrame の作成
df = pd.DataFrame(data_samples)
df.columns = ['物件名','アクセス1','アクセス2','所在地','築年数','建物の階数','賃料', '管理費', '敷金','礼金','仲介金','部屋の階数', '間取り', '面積', '方角']

df

,物件名,アクセス1,アクセス2,所在地,築年数,建物の階数,賃料,管理費,敷金,礼金,仲介金,部屋の階数,間取り,面積,方角
0,【空室14件】 ジェノヴィア池袋本町,東武東上線 下板橋駅 徒歩8分,東武東上線 北池袋駅 徒歩9分,東京都豊島区池袋本町２丁目,2023年8月(新築),7階建,"119,000円","管理費10,000円",無料,無料,無料,603号室,2K,25.95㎡,西
1,【空室14件】 ジェノヴィア池袋本町,東武東上線 下板橋駅 徒歩8分,東武東上線 北池袋駅 徒歩9分,東京都豊島区池袋本町２丁目,2023年8月(新築),7階建,"115,500円","管理費10,000円",無料,無料,無料,103号室,2K,25.95㎡,西
2,【空室14件】 ジェノヴィア池袋本町,東武東上線 下板橋駅 徒歩8分,東武東上線 北池袋駅 徒歩9分,東京都豊島区池袋本町２丁目,2023年8月(新築),7階建,"115,500円","管理費10,000円",無料,無料,無料,105号室,2K,25.95㎡,西
3,【空室1件】 パークアクシス池上,東急池上線 池上駅 徒歩7分,東急多摩川線 武蔵新田駅 徒歩11分,東京都大田区池上７丁目,2015年3月(築8年),5階建,"157,000円","管理費15,000円",15.7万円,無料,無料,306号室,2DK,41.12㎡,南
4,【空室1件】 オープンレジデンシア上野入谷,東京メトロ日比谷線 入谷駅 徒歩6分,つくばエクスプレス 浅草駅 徒歩9分,東京都台東区入谷２丁目,2023年8月(新築),15階建,"189,000円","管理費10,000円",18.9万円,18.9万円,"88,000円",301号室,2LDK,48.1㎡,西
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,【空室1件】 第一金子ビル,東京メトロ千代田線 綾瀬駅 徒歩5分,つくばエクスプレス 青井駅 徒歩12分,東京都足立区綾瀬４丁目,1985年1月(築38年),5階建,"90,000円","管理費5,000円",9.0万円,9.0万円,"64,000円",202号室,2DK,48.6㎡,西
362,【空室1件】 Ｂｅａｎ’ｓ新中野,東京メトロ丸ノ内線 新中野駅 徒歩2分,ＪＲ総武線 中野駅 徒歩14分,東京都中野区中央３丁目,2002年1月(築21年),3階建,"138,000円","管理費6,000円",13.8万円,13.8万円,"88,000円",207号室,2DK,38㎡,西
363,【空室1件】 ガーデンハイツたまがわ Ａ棟,東急大井町線 尾山台駅 徒歩20分,東急東横線 田園調布駅 徒歩18分,東京都世田谷区玉堤１丁目,1988年9月(築35年),2階建,"135,000円",管理費0円,13.5万円,13.5万円,"88,000円",A3号室,2LDK,57.96㎡,南西
364,【空室1件】 水元ハウス,ＪＲ常磐線 金町駅 バス8分 水元総合スポーツセンター入口下車 徒歩2分,京成金町線 京成金町駅 バス8分 水元総合スポーツセンター入口下車 徒歩2分,東京都葛飾区水元３丁目,1971年1月(築52年),2階建,"133,000円","管理費5,000円",26.6万円,13.3万円,"88,000円",1号室,2LDK,53㎡,北


In [192]:
#Step3
#出力した結果をCSVファイルに保存する
goodroom_df = df
goodroom_df.to_csv('airdoor_data.csv', index=False, encoding='utf-8-sig')